In [1]:
#| label: stock-sim
#| fig-cap: >
#|  Stock simulation where growth rate is normally distributed.

from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.models.callbacks import CustomJS
import numpy as np

output_notebook()

# Function to calculate accrued amount
def calculate_accrued_amount(starting_amount, monthly_investment, interest_rate_mean, interest_rate_std_dev, years):
    months = years * 12
    amount = starting_amount
    amounts = [amount]
    for month in range(1, months + 1):
        amount += monthly_investment
        interest_rate = np.random.normal(interest_rate_mean, interest_rate_std_dev)
        amount *= (1 + interest_rate / 12)
        amounts.append(amount)
    return amounts

# Initial parameters
starting_amount = 2000
monthly_investment = 200
years = 10
interest_rate_mean = 0.2
interest_rate_std_dev = 0.05

# Calculate initial data
amounts = calculate_accrued_amount(starting_amount, monthly_investment, interest_rate_mean, interest_rate_std_dev, years)
months = list(range(len(amounts)))

# Create ColumnDataSource
source = ColumnDataSource(data=dict(months=months, amounts=amounts))

# Create plot
plot = figure(title="Investment Growth Over Time", x_axis_label='Months', y_axis_label='Amount ($)')
plot.line('months', 'amounts', source=source, line_width=2)

# Create widgets
starting_amount_slider = Slider(start=0, end=10000, value=starting_amount, step=100, title="Starting Amount ($)")
monthly_investment_slider = Slider(start=0, end=5000, value=monthly_investment, step=50, title="Monthly Investment ($)")
years_slider = Slider(start=1, end=50, value=years, step=1, title="Years")
interest_rate_mean_slider = Slider(start=-1, end=1, value=interest_rate_mean, step=0.01, title="Interest Rate Mean")
interest_rate_std_dev_slider = Slider(start=0, end=1, value=interest_rate_std_dev, step=0.01, title="Interest Rate Std Dev")

# Callback function
callback = CustomJS(args=dict(source=source, 
                              starting_amount_slider=starting_amount_slider, 
                              monthly_investment_slider=monthly_investment_slider,
                              interest_rate_mean_slider=interest_rate_mean_slider,
                              interest_rate_std_dev_slider=interest_rate_std_dev_slider,
                              years_slider=years_slider), 
                    code="""
    function gaussianRandom(mean, stdDev) {
        let u1 = Math.random();
        let u2 = Math.random();
        let z0 = Math.sqrt(-2.0 * Math.log(u1)) * Math.cos(2.0 * Math.PI * u2);
        return z0 * stdDev + mean;
    }

    function calculate_accrued_amount(starting_amount, monthly_investment, interest_rate_mean, interest_rate_std_dev, years) {
        let months = years * 12;
        let amount = starting_amount;
        let amounts = [amount];
        
        for (let month = 1; month <= months; month++) {
            amount += monthly_investment;
            let interest_rate = gaussianRandom(interest_rate_mean, interest_rate_std_dev);
            amount *= (1 + interest_rate / 12);
            amounts.push(amount);
        }
        return amounts;
    }

    let starting_amount = starting_amount_slider.value;
    let monthly_investment = monthly_investment_slider.value;
    let interest_rate_mean = interest_rate_mean_slider.value;
    let interest_rate_std_dev = interest_rate_std_dev_slider.value;
    let years = years_slider.value;

    let amounts = calculate_accrued_amount(starting_amount, monthly_investment, interest_rate_mean, interest_rate_std_dev, years);
    let months = Array.from({length: amounts.length}, (_, i) => i);

    source.data = {months: months, amounts: amounts};
    source.change.emit();
""")

# Attach callback to widgets
starting_amount_slider.js_on_change('value', callback)
monthly_investment_slider.js_on_change('value', callback)
years_slider.js_on_change('value', callback)
interest_rate_mean_slider.js_on_change('value', callback)
interest_rate_std_dev_slider.js_on_change('value', callback)

# Layout
layout = column(plot, starting_amount_slider, monthly_investment_slider, years_slider, interest_rate_mean_slider, interest_rate_std_dev_slider)

# Show plot
show(layout)

Loading BokehJS ...